In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import ee
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import scipy
from scipy import stats
from scipy.stats import pearsonr
import time
import cartopy.crs as ccrs
import geopandas
import cartopy.io.img_tiles as cimgt
import warnings
warnings.filterwarnings("ignore")

stamen_terrain = cimgt.Stamen('terrain-background')
ll_proj = ccrs.PlateCarree()

ee.Initialize()

In [2]:
def GEE(date_start, date_stop, image_coll, patch, band2): 
    im_coll = dataset = ee.ImageCollection(image_coll).filterDate(ee.Date(date_start),ee.Date(date_stop)).select(band2)

    acq_times = im_coll.aggregate_array('system:time_start').getInfo()
    avalaible_dates = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
    im_list = im_coll.toList(im_coll.size())
    
    variable_list = ee.List([])
   
    def reduce_dataset_region(image, list):
        local_image = image.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=patch,
            scale=1,
            crs = "epsg:4326")

        return ee.List(list).add(local_image)
    
    reduced_dataset = im_coll.iterate(reduce_dataset_region, variable_list)
    
    reduced_dataset_dict = reduced_dataset.getInfo()
    df1 = pd.DataFrame(data = reduced_dataset_dict, index = avalaible_dates, columns = [band2]).dropna()
    df1.index = pd.to_datetime(df1.index)
    return df1

In [7]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
# GEE(date_start, date_stop, image_coll, patch, band2)

In [29]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
point = ee.Geometry.Point([23, 40])
dataseries = []
for i in months:
    dataseries.append(ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").select(i).reduceRegion(ee.Reducer.median(), point).getInfo()[i])
    
pd.DataFrame(dataseries)

,0
0,45
1,27
2,29
3,22
4,46
5,41
6,37
7,41
8,115
9,61
